In [1]:
# imports
import os

import numpy as np
import pandas as pd

In [2]:
# read excel file

directory = '/Users/beekayokay/OneDrive/Projects/Salesforce Epicor Integration/'
file_name = 'test_bookings_df.xlsx'

In [3]:
bookings_df = pd.read_excel(
    os.path.join(directory, file_name),
    dtype={
        'Book Date': 'datetime64[D]',
        'Need By Date': 'datetime64[D]',
        'Quote Num': 'object',
        'Quote Line': 'object'
    }
)

In [31]:
bookings_df.head()

,Tran Num,Book Date,BookType,OrderNum,OrderLn,OrderRel,Prod Code,SN,Order Line Amt,PO Num,...,ShipTo.Address3,ShipTo.City,ShipTo.State,ShipTo.ZIP,ShipTo.Country,Packager,Market Segment,Payment Terms,OrderHed.FOB,SysRowID
0,4649,2003-10-02,New,1000,1,1,AIIE,E110042-3,99000.0,A009-41003 R1,...,NaN,NaN,NaN,NaN,INDONESIA,NaN,NaN,XLETTER OF CRED,C&F JAKARTA,92059e56-e620-40ac-e111-e6c2e1812162
1,4650,2003-12-30,Void,1000,1,1,AIIE,E110042-3,-99000.0,A009-41003 R1,...,NaN,NaN,NaN,NaN,INDONESIA,NaN,NaN,XLETTER OF CRED,C&F JAKARTA,92059e56-e620-40ac-e111-e6c2e2812162
2,4651,2003-10-02,New,1001,1,1,AIKE,E211900B,496000.0,02USSHY26SPEC2201,...,NaN,BEIJING,NaN,100101,CHINA,NaN,INDUS,NET 45 DAYS,FOB NY PORT,92059e56-e620-40ac-e111-e6c2e3812162
3,4652,2003-10-02,New,1002,1,1,AICE,E311603B-1,962000.0,SMJR2BOO11,...,NaN,AL-JUBAIL,NaN,31961,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2e4812162
4,4653,2003-12-19,Chg,1002,1,1,AICE,E311603B-1,-962000.0,SMJR2BOO11,...,NaN,AL-JUBAIL,NaN,31961,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2e5812162


In [35]:
# sort and modify values

bookings_df.sort_values(
    by=['OrderNum', 'OrderLn', 'OrderRel'],
    ascending=True,
    inplace=True
)

bookings_df.loc[bookings_df['Quote Num'] == 0, 'Quote Num'] = np.nan
bookings_df.loc[bookings_df['Quote Line'] == 0, 'Quote Line'] = np.nan

In [62]:
df1 = bookings_df[['OrderNum', 'OrderLn', 'OrderRel']].copy()
df1.drop_duplicates(
    subset=['OrderNum', 'OrderLn'],
    keep='last', inplace=True,
)
df1.shape

(224475, 3)

In [63]:
df2 = bookings_df[['OrderNum', 'OrderLn', 'OrderRel']].copy()
df2.drop_duplicates(inplace=True, keep='first')
df2 = df2.groupby(
    ['OrderNum', 'OrderLn'],
    as_index=False
).agg(
    {'OrderRel': 'count'}
)
df2.shape

(224475, 3)

In [64]:
df1.to_excel('df1.xlsx')
df2.to_excel('df2.xlsx')

In [26]:
# check current

bookings_df.shape

(345868, 43)

In [27]:
# deduped order/line check for later

order_line_df = bookings_df[['OrderNum', 'OrderLn']]
order_line_df.drop_duplicates(inplace=True)
order_line_df.shape

(224475, 2)

In [28]:
# take out duplicated rows due to OrderRel field

dup_ind = bookings_df[['Tran Num', 'OrderNum', 'OrderLn']].duplicated(keep='first')
print(f'Duplicated Rows: {len(bookings_df.loc[dup_ind,:])}')
print(f'Deduped Rows: {len(bookings_df.loc[~dup_ind,:])}')
bookings_df = bookings_df.loc[~dup_ind,:]

Duplicated Rows: 11572
Deduped Rows: 334296


In [29]:
# modify DataFrame and preview dtypes
bookings_df.sort_values(
    by=['Tran Num'],
    ascending=True,
    inplace=True
)

In [48]:
bookings_df.rename(
    columns = {
        'Tran Num': 'Name',
        'Book Date': 'Book_Date__c',
        'BookType': 'BookType__c',
        'OrderNum': 'OrderNum__c',
        'OrderLn': 'OrderLn__c',
        'OrderRel': 'OrderRel__c',
        'Prod Code': 'Prod_Code__c',
        'SN': 'SN__c',
        'Order Line Amt': 'Order_Line_Amt__c',
        'PO Num': 'PO_Num__c',
        'Need By Date': 'Need_By_Date__c',
        'Quote Num': 'Quote_Num__c',
        'Quote Line': 'Quote_Line__c',
        'Sales Representative': 'Sales_Representative__c',
        'Salesperson': 'Salesperson__c',
        'Primary Salesperson Origin': 'Primary_Salesperson_Origin__c',
        'Sales Director Origin': 'Sales_Director_Origin__c',
        'Regional Sales Manager Origin': 'Regional_Sales_Manager_Origin__c',
        'Cust. ID': 'Cust_ID__c',
        'Customer Name': 'Customer_Name__c',
        'Ultimate User': 'Ultimate_User__c',
        'Bill To Name': 'Bill_To_Name__c',
        'Bill To  Address': 'Bill_To_Address_1__c',
        'Bill To Address2': 'Bill_To_Address_2__c',
        'Bill To Address3': 'Bill_To_Address_3__c',
        'Bill To  Address.1': 'Bill_To_City__c',
        'Bill To State': 'Bill_To_State__c',
        'Bill To Zip': 'Bill_To_Zip__c',
        'Bill To Country': 'Bill_To_Country__c',
        'ShipToNum': 'Ship_To_Num__c',
        'ShipTo.Name': 'Ship_To_Name__c',
        'ShipTo.Address1': 'Ship_To_Address_1__c',
        'ShipTo.Address2': 'Ship_To_Address_2__c',
        'ShipTo.Address3': 'Ship_To_Address_3__c',
        'ShipTo.City': 'Ship_To_City__c',
        'ShipTo.State': 'Ship_To_State__c',
        'ShipTo.ZIP': 'Ship_To_Zip__c',
        'ShipTo.Country': 'Ship_To_Country__c',
        'Packager': 'Packager__c',
        'Market Segment': 'Market_Segment__c',
        'Payment Terms': 'Payment_Terms__c',
        'OrderHed.FOB': 'OrderHed_FOB__c',
        'SysRowID': 'SysRowID__c'
    },
    inplace = True
)

In [53]:
bookings_df_grouped = bookings_df.groupby(
    ['OrderNum__c', 'OrderLn__c']
).agg({
    'Name': 'last',
    'Book_Date__c': 'last',
    'BookType__c': 'last',
    'OrderRel__c': 'last',
    'Prod_Code__c': 'last',
    'SN__c': 'first',
    'Order_Line_Amt__c': 'sum',
    'PO_Num__c': 'first',
    'Need_By_Date__c': 'last',
    'Quote_Num__c': 'first',
    'Quote_Line__c': 'first',
    'Sales_Representative__c': 'first',
    'Salesperson__c': 'first',
    'Primary_Salesperson_Origin__c': 'first',
    'Sales_Director_Origin__c': 'first',
    'Regional_Sales_Manager_Origin__c': 'first',
    'Cust_ID__c': 'first',
    'Customer_Name__c': 'first',
    'Ultimate_User__c': 'first',
    'Bill_To_Name__c': 'first',
    'Bill_To_Address_1__c': 'first',
    'Bill_To_Address_2__c': 'first',
    'Bill_To_Address_3__c': 'first',
    'Bill_To_City__c': 'first',
    'Bill_To_State__c': 'first',
    'Bill_To_Zip__c': 'first',
    'Bill_To_Country__c': 'first',
    'Ship_To_Num__c': 'first',
    'Ship_To_Name__c': 'first',
    'Ship_To_Address_1__c': 'first',
    'Ship_To_Address_2__c': 'first',
    'Ship_To_Address_3__c': 'first',
    'Ship_To_City__c': 'first',
    'Ship_To_State__c': 'first',
    'Ship_To_Zip__c': 'first',
    'Ship_To_Country__c': 'first',
    'Packager__c': 'first',
    'Market_Segment__c': 'first',
    'Payment_Terms__c': 'first',
    'OrderHed_FOB__c': 'first',
    'SysRowID__c': 'last'
})

In [54]:
bookings_df_grouped

Name Book_Date__c BookType__c  OrderRel__c  \
OrderNum__c OrderLn__c                                                 
1000        1             4650   2003-12-30        Void            1   
1001        1             4651   2003-10-02         New            1   
1002        1             4654   2003-12-19         Chg            1   
            2             4659   2003-12-19         Chg            1   
            3             4662   2003-12-19         Chg            1   
...                        ...          ...         ...          ...   
68348       1           761047   2020-02-28         New            1   
68349       1           761048   2020-02-28         New            1   
            2           761049   2020-02-28         New            1   
            3           761050   2020-02-28         New            1   
            4           761051   2020-02-28         New            1   

                       Prod_Code__c        SN__c  Order_Line_Amt__c  \
OrderNum__c OrderLn__c                                                
1000        1                  AIIE    E110042-3               0.00   
1001        1                  AIKE     E211900B          496000.00   
1002        1                  AICE   E311603B-1          320666.67   
            2                  AICE   E311603B-2          320666.67   
            3                  AICE   E311603B-3          320666.66   
...                             ...          ...                ...   
68348       1                  PIDE  B5EU2E039-1            1836.78   
68349       1                  PIDA     A8ET2116           70273.32   
            2                  PIDE     A8ET2116              70.38   
            3                  PIDE     A8ET2116              70.38   
            4                  PIDE     A8ET2116             182.16   

                                PO_Num__c Need_By_Date__c  Quote_Num__c  ...  \
OrderNum__c OrderLn__c                                                   ...   
1000        1               A009-41003 R1      2002-06-05           NaN  ...   
1001        1           02USSHY26SPEC2201      2003-09-01           NaN  ...   
1002        1                  SMJR2BOO11      2003-10-17           NaN  ...   
            2                  SMJR2BOO11      2003-10-17           NaN  ...   
            3                  SMJR2BOO11      2003-10-17           NaN  ...   
...                                   ...             ...           ...  ...   
68348       1                      044061      2020-03-20       61711.0  ...   
68349       1                      044062      2020-03-27           NaN  ...   
            2                      044062      2020-03-27           NaN  ...   
            3                      044062      2020-03-27           NaN  ...   
            4                      044062      2020-03-27           NaN  ...   

                        Ship_To_Address_3__c Ship_To_City__c Ship_To_State__c  \
OrderNum__c OrderLn__c                                                          
1000        1                            NaN             NaN              NaN   
1001        1                            NaN         BEIJING              NaN   
1002        1                            NaN       AL-JUBAIL              NaN   
            2                            NaN       AL-JUBAIL              NaN   
            3                            NaN       AL-JUBAIL              NaN   
...                                      ...             ...              ...   
68348       1                            NaN      GREENSBORO               NC   
68349       1                            NaN          MONROE               NC   
            2                            NaN          MONROE               NC   
            3                            NaN          MONROE               NC   
            4                            NaN          MONROE               NC   

                       Ship_To_Zip__c Ship_To_Country__c Packager__c  

In [55]:
bookings_df_grouped.to_csv(os.path.join(directory, 'group_test.csv'))